In [26]:
from mlflow.tracking import MlflowClient
import mlflow

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

In [27]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/398662229275304820', creation_time=1747326044570, experiment_id='398662229275304820', last_update_time=1747326044570, lifecycle_stage='active', name='my-new-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/705382483651804432', creation_time=1747325141755, experiment_id='705382483651804432', last_update_time=1747325141755, lifecycle_stage='active', name='my-experiment-2', tags={}>,
 <Experiment: artifact_location='file:///c:/ML/cpe393-mlflow/mlruns/938325151512188093', creation_time=1747321085741, experiment_id='938325151512188093', last_update_time=1747321085741, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///c:/ML/cpe393-mlflow/mlruns/0', creation_time=1747321078030, experiment_id='0', last_update_time=1747321078030, lifecycle_stage='active', name='Default', tags={}>]

In [30]:
client.create_experiment(name="my-new-experiment2")

'142066508938646353'

In [31]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='',
    filter_string="metrics.rmse < 100",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [32]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

In [33]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
run_id = "0c7832b42c4e4779a759b099ea548adf"  #insert your run id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/05/15 23:27:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1747326475248, current_stage='None', description='', last_updated_timestamp=1747326475248, name='nyc-taxi-regressor', run_id='0c7832b42c4e4779a759b099ea548adf', run_link='', source='file:///c:/ML/cpe393-mlflow/mlruns/938325151512188093/0c7832b42c4e4779a759b099ea548adf/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='3'>

In [35]:
#check model version and stages
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None


C:\Users\intou\AppData\Local\Temp\ipykernel_38628\1279244624.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [36]:
model_version = 4
new_stage = "Staging"
# Use client.transition_model_version_stage() function to model to staging.

In [37]:

import pandas as pd

# Load March 2021 dataset (you can increase nrows if needed)
df = pd.read_csv("C:\ML\cpe393-mlflow\green_tripdata_2019-03.csv", nrows=10000)

# Feature Engineering
df['pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
df['duration'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds() / 60
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Convert categorical to string
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

# Convert to dict for vectorizer
dicts = df[categorical].to_dict(orient='records')


<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
C:\Users\intou\AppData\Local\Temp\ipykernel_38628\2873031252.py:4: SyntaxWarning: invalid escape sequence '\M'
  df = pd.read_csv("C:\ML\cpe393-mlflow\green_tripdata_2019-03.csv", nrows=10000)


In [42]:
import mlflow
import pickle

# Replace with actual run_id if known, or use MLflow UI to find it
# Example path - adjust this
vectorizer_path = mlflow.artifacts.download_artifacts(run_id="0c7832b42c4e4779a759b099ea548adf")

with open(vectorizer_path, "rb") as f:
    dv = pickle.load(f)

# Transform test set
X_test = dv.transform(dicts)

PermissionError: [Errno 13] Permission denied: 'C:\\ML\\cpe393-mlflow\\mlruns\\938325151512188093\\0c7832b42c4e4779a759b099ea548adf\\artifacts'